### Prepare data for training and save them to disk.

In [1]:
from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import os
import numpy as np
import pandas as pd
import glob
import math

In [2]:
from process_data import data_uID

In [10]:
path='input/train_1'

In [4]:
data = data_uID()

In [5]:
def check_files(path):
    train = np.unique([p.split('-')[0] for p in sorted(glob.glob(path + '/**'))])
    for event in train:
        try: 
            hits, cells, particles, truth = load_event(event)
        except:
            print("reading event:", event)

In [6]:
data.load_training(path='input/train_1', eta_cut=3.5)

In [10]:
for key in data.event_list.keys():
    post_fix = ".pkl.gz"
    value = data.event_list[key]
    value[0].to_pickle(key+"_filtered_hits"+post_fix)
    value[1].to_pickle(key+"_filtered_particles"+post_fix)

In [12]:
total_particles = sum(x[0].shape[0] for x in data.event_list.values())
total_hits = sum(x[1].shape[0] for x in data.event_list.values())

In [13]:
print(total_particles)
print(total_hits)

18134379
1474386


In [4]:
from utils import get_features
import pickle
def ten_hits_data(path, out_name):
    train = np.unique([p.split('-')[0] for p in sorted(glob.glob(path + '/**'))])
    event_list = []
    for event in train:
        try: 
            hits, cells, particles, truth = load_event(event)
            pIDs = particles[particles['nhits'] == 10]['particle_id']
            hits_truth = pd.merge(hits, truth, on=['hit_id'])
            hits_truth = get_features(hits_truth)
            
            track_list = []
            for pID in pIDs:
                if pID == 0:
                    continue
                this_track = hits_truth[hits_truth['particle_id'] == pID][['r', 'phi', 'z']].values
#                 this_track = hits_truth[(hits_truth['particle_id'] == pID) & (hits_truth['eta'] > 1) & (hits_truth['eta'] > -1)][['r', 'phi', 'z']].values
                track_list.append(this_track)
                
            event_list.append(track_list)
        except:
            print("reading event:", event)
            
    with open(out_name, 'wb') as fp:
        pickle.dump(event_list, fp)
    return event_list

In [ ]:
events = ten_hits_data('input/train_1', 'ten_hists.npy')

In [11]:
with open('ten_hists.npy', 'wb') as fp:
    pickle.dump(events, fp)

In [ ]:
events = ten_hits_data('input/train_1', 'ten_hists_eta_less_1.npy')

In [ ]:
def training_data_with_eta_cut(train_dir='input/train_1', event_prefix="event000001000", eta_cut=3.2):
    hits, cells, particles, truth = load_event(os.path.join(train_dir, event_prefix))

    hits_features = get_features(hits)
#     high_eta_hits = hits_features[(hits_features['eta'] > eta_cut) | (hits_features['eta'] < -1 * eta_cut)]
    high_eta_hits = hits_features[(hits_features['eta'] > eta_cut) | (hits_features['eta'] < -1 * eta_cut)]
    uID_for_higheta = make_uID(high_eta_hits)
    high_eta_hits_uID = pd.merge(high_eta_hits, uID_for_higheta, on=['volume_id', 'layer_id', 'module_id'])
    train_data_higheta = high_eta_hits_uID.merge(filter_truth(truth), on='hit_id')[['uID', 'particle_id']]

    return train_data_higheta, uID_for_higheta.shape[0]